In [ ]:
#提取数据
import numpy as np  # 用于数学计算和数组操作
import pandas as pd  # 用于数据处理和分析
import matplotlib.pyplot as plt  # 用于数据可视化
import polars as pl  # 高性能数据处理库，类似于pandas
import datetime  # 用于处理日期和时间
from tqdm import tqdm  # 用于显示进度条
import plotly.express as px  # 用于交互式数据可视化
from plotly.subplots import make_subplots  # 用于创建子图
import plotly.graph_objects as go  # 用于创建更复杂的图表
from sklearn.model_selection import train_test_split  # 用于划分训练集和测试集
import sys
sys.path.append('/home/zhuangzhuohan/sleep_data')
from my_py.my_funs import make_train_dataset,load_and_preprocess_data,create_features,apply_features,analyze_validation_data,analyze_val_data_stats

tolerances = {
    'onset': [12, 36, 60, 90, 120, 150, 180, 240, 300, 360],  # 睡眠开始事件的时间容忍度（分钟）
    'wakeup': [12, 36, 60, 90, 120, 150, 180, 240, 300, 360]  # 睡眠结束事件的时间容忍度（分钟）
}
# 1. 加载和预处理数据
train_series, train_events, test_series ,series_ids = load_and_preprocess_data()
# 2. 创建特征
features, feature_cols, id_cols = create_features()
# 3. 应用特征到训练数据
train_series = apply_features(train_series, features, id_cols, feature_cols)
test_series = apply_features(test_series, features, id_cols, feature_cols)
# 2. 定义列名映射（用于评分函数）
column_names = {
    'series_id_column_name': 'series_id',
    'time_column_name': 'step',
    'event_column_name': 'event',
    'score_column_name': 'score',
}
# 3. 划分训练集和验证集（70%训练，30%验证）
train_ratio = 0.7
random_seed = 42
train_ids, val_ids = train_test_split(series_ids, train_size=train_ratio, random_state=random_seed)
# 4. 收集训练数据，每5分钟取一个数据点（减少数据量）
train_data = train_series.filter(pl.col('series_id').is_in(train_ids)).collect()
# 转换为pandas DataFrame后使用切片方法
step_time = 12 * 5  # 每5分钟取一个数据点
train_data = train_data.to_pandas().iloc[::(step_time)]  # 每step_time分钟取一个数据点
train_data = pl.from_pandas(train_data)  # 转回polars DataFrame
# 创建训练事件数据（只包含训练集的事件）
train_solution_series_id = train_events.filter(pl.col('series_id').is_in(train_ids))
train_solution = train_events.filter(pl.col('series_id').is_in(train_ids)).select(['series_id', 'event', 'step']).to_pandas()
# 统计真实的onset和wakeup事件数量
train_solution_onset_count = len(train_solution[train_solution['event'] == 'onset'])
train_solution_wakeup_count = len(train_solution[train_solution['event'] == 'wakeup'])
print(f"train_solution的onset事件数量: {train_solution_onset_count}")
print(f"train_solution的wakeup事件数量: {train_solution_wakeup_count}")
# 创建训练数据集
X_train, y_train = make_train_dataset(train_data, train_solution_series_id, feature_cols, id_cols)
# 5. 创建验证数据
val_data = train_series.filter(pl.col('series_id').is_in(val_ids)).collect()
# 6. 创建验证标签（用于评估模型性能）
val_solution = train_events.filter(pl.col('series_id').is_in(val_ids)).select(['series_id', 'event', 'step']).to_pandas()
# 获取验证集的完整事件数据（包含night和timestamp列）
val_events = train_events.filter(pl.col('series_id').is_in(val_ids))
# 分析验证数据统计信息
analyze_val_data_stats(val_data, val_events)
# 统计验证数据并保存到CSV文件
analyze_validation_data(
    val_data=val_data,
    val_ids=val_ids,
    val_solution=val_solution,
    train_events=train_events,
)

In [ ]:
'''
import polars as pl
from sklearn.model_selection import train_test_split
# 加载train_events获取series_ids
train_events = pl.read_csv('/home/zhuangzhuohan/sleep_data/train_events/train_events.csv')
series_ids = train_events['series_id'].unique().to_list()
# 只选择10个series_id
top_my_val_ids = val_ids[:1]
print(f"选择的1个series_id: {top_my_val_ids}")
print(f"验证集包含 {len(top_my_val_ids)} 个series_id")
# 加载train_series.parquet并提取验证集数据
train_series_timezone = pl.read_parquet('/home/zhuangzhuohan/sleep_data/train_series/train_series.parquet')
val_timezone = train_series_timezone.filter(pl.col('series_id').is_in(top_my_val_ids))
val_timezone.to_pandas().to_csv('/home/zhuangzhuohan/sleep_data/val/val_timezone.csv', index=False)
print(f"val_timezone.csv 保存完成，共 {len(val_timezone)} 行数据")
# 加载train_series_no_timezone.parquet并提取验证集数据
train_series_no_timezone = pl.read_parquet('/home/zhuangzhuohan/sleep_data/train_series/train_series_no_timezone.parquet')
val_no_timezone = train_series_no_timezone.filter(pl.col('series_id').is_in(top_my_val_ids))
val_no_timezone.to_pandas().to_csv('/home/zhuangzhuohan/sleep_data/val/val_no_timezone.csv', index=False)
print(f"val_no_timezone.csv 保存完成，共 {len(val_no_timezone)} 行数据")
# 从train_events_clean.csv中提取验证集数据
print("处理train_events_clean.csv...")
train_events_clean = pl.read_csv('/home/zhuangzhuohan/sleep_data/train_events/train_events_clean.csv')
val_event_timezone = train_events_clean.filter(pl.col('series_id').is_in(top_my_val_ids))
val_event_timezone.to_pandas().to_csv('/home/zhuangzhuohan/sleep_data/val/val_event_timezone.csv', index=False)
print(f"val_event_timezone.csv 保存完成，共 {len(val_event_timezone)} 行数据")
# 从train_events_clean_no_timezone.csv中提取验证集数据
print("处理train_events_clean_no_timezone.csv...")
train_events_clean_no_timezone = pl.read_csv('/home/zhuangzhuohan/sleep_data/train_events/train_events_clean_no_timezone.csv')
val_event_no_timezone = train_events_clean_no_timezone.filter(pl.col('series_id').is_in(top_my_val_ids))
val_event_no_timezone.to_pandas().to_csv('/home/zhuangzhuohan/sleep_data/val/val_event_no_timezone.csv', index=False)
print(f"val_event_no_timezone.csv 保存完成，共 {len(val_event_no_timezone)} 行数据")
'''

In [ ]:
'''
import pandas as pd

# 查看CSV文件的前几行
df = pd.read_csv('/home/zhuangzhuohan/sleep_data/val/val_no_timezone.csv')
print("文件前5行:")
print(df.head())
print("\n文件信息:")
print(df.info())
print("\n数据类型:")
print(df.dtypes)
'''

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# 定义模型超参数字典
model_params = {
    'n_estimators': 25,              # 决策树数量（默认：100）
    'criterion': 'gini',             # 分裂标准（默认：'gini'）
    'max_depth': None,                 # 树的最大深度（默认：None）
    'min_samples_split': 2,          # 分裂所需的最小样本数（默认：2）
    'min_samples_leaf': 100,         # 每个叶节点最少样本数（默认：1）
    'min_weight_fraction_leaf': 0.0, # 叶节点最小权重分数（默认：0.0）
    'max_features': None,          # 寻找最佳分裂时考虑的特征数量（默认：'sqrt'）
    'max_leaf_nodes': None,          # 最大叶节点数（默认：None）
    'min_impurity_decrease': 0,    # 最小不纯度减少（默认：0.0）
    'bootstrap': True,               # 是否使用bootstrap抽样（默认：True）
    'oob_score': False,              # 是否使用袋外样本评估（默认：False）
    'n_jobs': -1,                    # 使用所有CPU核心（默认：None）
    'random_state': random_seed,     # 随机种子（默认：None）
    'verbose': 0,                    # 详细程度（默认：0）
    'warm_start': False,             # 是否使用上次的拟合结果作为起点（默认：False）
    'class_weight':{0: 1, 1: 3},      # 类别权重（默认：None）
    'ccp_alpha': 0.0,                # 剪枝参数（默认：0.0）
    'max_samples': None,             # bootstrap抽样的最大样本数（默认：None）
    'monotonic_cst': None            # 单调约束（默认：None）
}
# 初始化随机森林分类器
rf_classifier = RandomForestClassifier(random_state=model_params['random_state'])
# 训练分类器（设置超参数）
rf_classifier = RandomForestClassifier(**model_params)
# 拟合模型
rf_classifier.fit(X_train[feature_cols], y_train)
# 绘制特征重要性图
px.bar(x=feature_cols, 
       y=rf_classifier.feature_importances_,  # 随机森林模型的特征重要性得分
       title='Random forest feature importances'
      )

In [ ]:
from my_py.metric import score  # 导入自定义的评分函数
from my_py.model_analyzer import analyze_sleep_model_performance
from my_py.my_funs import get_events
# 在验证集上检查模型性能
min_sleep_duration = 12*30
rf_submission = get_events(val_data, rf_classifier, feature_cols, id_cols, min_sleep_duration)  # 生成验证集的预测事件
# 调用函数，传入 rf_classifier 参数以计算内存
performance_results = analyze_sleep_model_performance(
    rf_submission=rf_submission,
    ground_truth=val_solution,  # 使用 val_solution 作为真实事件数据
    val_data=val_data,
    val_ids=val_ids,
    rf_classifier=rf_classifier,  # 传入训练好的分类器模型
    min_sleep_duration=min_sleep_duration  # 可根据需要调整最小睡眠周期长度
)
# 计算模型得分
rf_score = score(val_solution, rf_submission, tolerances, **column_names)
print(f"Random forest score: {rf_score}")

In [ ]:
'''
# 导入保存结果函数
from my_py.save_model_results import save_model_results
# 保存模型结果，传入 performance_results 参数
save_model_results(
    n_estimators=model_params['n_estimators'],
    min_samples_leaf=model_params['min_samples_leaf'],
    random_seed=model_params['random_state'],
    min_sleep_cycle='12*30(6h)',
    new_window_feature_time='None',#class_weight
    train_sampling_freq='12*5(5分钟)',
    train_set_ratio=0.7,
    test_content='',
    score=rf_score,
    performance_results=performance_results  # 传递性能结果
)
'''

In [ ]:
import pickle
import os
import onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
# 定义保存路径
pkl_save_path = '/home/zhuangzhuohan/sleep_data/sleep_data/sleep_data.pkl'
onnx_save_path = '/home/zhuangzhuohan/sleep_data/sleep_data/sleep_data.onnx'
# 确保目录存在
os.makedirs(os.path.dirname(pkl_save_path), exist_ok=True)
os.makedirs(os.path.dirname(onnx_save_path), exist_ok=True)
# 保存模型
with open(pkl_save_path, 'wb') as f:
    pickle.dump(rf_classifier, f)
# 定义输入形状
initial_type = [('float_input', FloatTensorType([None, len(feature_cols)]))]
# 转换模型
onnx_model = convert_sklearn(rf_classifier, initial_types=initial_type)
# 保存ONNX模型
onnx.save_model(onnx_model, onnx_save_path)

In [ ]:
from my_py.save_model_results_new import save_model_results_new

row_number = save_model_results_new(
        model_params=model_params,
        rf_score=rf_score,
        rf_submission=rf_submission,
        performance_results=performance_results,
        train_ratio=train_ratio,
        step_time=step_time,
        min_sleep_duration=min_sleep_duration
)
print(f"测试成功！数据已保存到第 {row_number} 行")